In [ ]:
from icecream import ic
from pprint import pprint

In [ ]:
text = "hello louis hey 🤐!!!! how is the world ?"
# text = "🤐"
# text = "hello world!"
# text = "é"
tokens = text.encode("utf-8")
tokens
int(tokens[0])
# len(list(map(int, tokens))), len(text)
print([int(x) for x in tokens])

In [ ]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


ids = [1, 2, 3, 4, 3, 4, 7, 8, 9, 10, 8, 9]
stats = get_stats(tokens)
# sorted(((v, k) for k, v in counts.items()), reverse=True)
stats

In [ ]:
top_pair = max(stats, key=stats.get)
top_pair

In [ ]:
def merge(ids, pair, new_idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            new_ids.append(new_idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1

    return new_ids


merge([1, 2, 5, 6, 7, 5, 6], (5, 6), 99)

merge(tokens, top_pair, 256)

In [ ]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)
merges = {}

for i in range(num_merges):
    stats = get_stats(ids)
    if len(stats) > 0:
        top_pair = max(stats, key=stats.get)
        idx = 256 + i
        ids = merge(ids, top_pair, idx)
        merges[top_pair] = idx
    else:
        break
merges

In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    # ic(p0, p1)
    vocab[idx] = vocab[p0] + vocab[p1]


def decode(ids: list) -> str:
    tokens = b"".join(vocab[idx] for idx in ids)
    txt = tokens.decode("utf-8", errors="replace")
    return txt

In [ ]:
merges

In [ ]:
tokens = list("world!!".encode("utf-8"))
stats = get_stats(tokens)
ic(stats)
min(stats, key=lambda p: merges.get(p, float("inf")))

In [ ]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    ic(tokens)
    while True:
        stats = get_stats(tokens)
        ic(stats)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        ic(pair)
        break


encode("Hello")

In [ ]:
tokens = list("hello world hey!!".encode("utf-8"))
stats = get_stats(tokens)
min(stats, key=lambda p: merges.get(p, float("inf")))

In [ ]:
stats
merges